# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)

In [1]:
import pandas as pd
import pymorphy2
import re

In [2]:
news_train = pd.read_csv(
    'news/news_train.txt',
    sep='\t',
    header=None,
    names=['label', 'header', 'text'],
)
news_test = pd.read_csv(
    'news/news_test.txt',
    sep='\t',
    header=None,
    names=['label', 'header', 'text'],
)

In [3]:
news_train

,label,header,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...
...,...,...,...
14995,life,Составлен рейтинг лучших европейских пляжей 20...,Опубликован рейтинг лучших европейских пляжей ...
14996,media,В «Снобе» объяснили причину смены формата,Генеральный директор «Сноб медиа» Марина Гевор...
14997,economics,Минфин предложил штрафовать за биткоины на 50 ...,"Минфин разработал законопроект, устанавливающи..."
14998,life,Мэл Гибсон заплатит бывшей подруге 750 тысяч д...,Актер и режиссер Мэл Гибсон выплатит своей быв...


In [4]:
news_test

,label,header,text
0,culture,Жительница Ямала победила в первом песенном ко...,Жительница Ямало-Ненецкого автономного округа ...
1,media,Почти половина Twitter-пользователей никогда н...,Около 44 процентов из всех зарегистрированных ...
2,media,"Билайн начал рекламу роуминга под песенку ""Тро...",В новой рекламной кампании мобильного оператор...
3,business,"Saipem потеряла 1,2 миллиарда евро из-за отмен...",Дочерняя структура итальянского нефтегазового ...
4,culture,Вин Дизель назвал «Форсаж 7» достойным «Оскара»,"Актер Вин Дизель заявил, что боевик «Форсаж 7»..."
...,...,...,...
2995,science,"Причиной ""влажного"" климата Титана оказались м...","Ученые прояснили причины ""влажного"" климата Ти..."
2996,life,Британка нашла геккона в пакете с брокколи,Жительница Великобритании нашла геккона в паке...
2997,business,Владелец «Мечела» предложил закрыть в России в...,Совладелец горно-металлургического холдинга «М...
2998,science,Nokia выпустит ОС для бюджетных смартфонов,Компания Nokia разрабатывает операционную сист...


In [5]:
analyzer = pymorphy2.MorphAnalyzer()

In [6]:
WORD_PATTERN = '(?u)\\b\\w\\w+\\b'  
reg_exp = re.compile(pattern=WORD_PATTERN)

In [7]:
news_train['tokens'] = [
    [analyzer.parse(word)[0].normal_form for word in reg_exp.findall(text.lower())]
    for text in news_train['text']
]

In [8]:
news_test['tokens'] = [
    [analyzer.parse(word)[0].normal_form for word in reg_exp.findall(text.lower())]
    for text in news_test['text']
]

## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

In [9]:
from gensim.models.word2vec import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

In [10]:
sentences = news_train['tokens'].values

In [11]:
class LossLogger(CallbackAny2Vec):
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss - self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [12]:
w2v_model = Word2Vec(min_count=1, sg=1, workers=8)
w2v_model.build_vocab(sentences)

In [13]:
w2v_model.train(
    corpus_iterable=sentences,
    total_examples=w2v_model.corpus_count,
    epochs=20,
    compute_loss=True,
    callbacks=[LossLogger()]
)

Loss after epoch 0: 4451133.0
Loss after epoch 1: 2972504.0
Loss after epoch 2: 2941971.0
Loss after epoch 3: 2842250.0
Loss after epoch 4: 2941605.0
Loss after epoch 5: 2335597.0
Loss after epoch 6: 2504094.0
Loss after epoch 7: 2360644.0
Loss after epoch 8: 2497062.0
Loss after epoch 9: 2184636.0
Loss after epoch 10: 2388058.0
Loss after epoch 11: 2462740.0
Loss after epoch 12: 1559406.0
Loss after epoch 13: 1322512.0
Loss after epoch 14: 1187244.0
Loss after epoch 15: 1232496.0
Loss after epoch 16: 1219600.0
Loss after epoch 17: 1247796.0
Loss after epoch 18: 1166628.0
Loss after epoch 19: 1148372.0


(49036353, 53022760)

In [14]:
w2v_model.wv.most_similar(positive=['футбол'], topn=5)

[('хоккей', 0.7785260677337646),
 ('сборная', 0.7456079721450806),
 ('чемпионат', 0.7218098044395447),
 ('чм', 0.715583324432373),
 ('фифа', 0.7150311470031738)]

In [15]:
w2v_model.wv.most_similar(positive=['фильм'], topn=5)

[('картина', 0.8522942066192627),
 ('полнометражный', 0.7678956389427185),
 ('режиссёр', 0.7641991376876831),
 ('неудержимый', 0.7224044799804688),
 ('полнометражка', 0.715556263923645)]

In [16]:
w2v_model.wv.most_similar(positive=['apple'], topn=5)

[('iphone', 0.8077396154403687),
 ('samsung', 0.7983869910240173),
 ('microsoft', 0.7740789651870728),
 ('смартфон', 0.7692180275917053),
 ('motorola', 0.7642343640327454)]

In [17]:
w2v_model.wv.most_similar(positive=['google'], topn=5)

[('microsoft', 0.8154850006103516),
 ('поисковик', 0.8139421939849854),
 ('bing', 0.7637169361114502),
 ('сервис', 0.7595903277397156),
 ('yahoo', 0.753369927406311)]

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [25]:
from sklearn.model_selection import GridSearchCV

In [21]:
X_train = [' '.join(tokens) for tokens in news_train['tokens']]
y_train = news_train['label']
X_test = [' '.join(tokens) for tokens in news_test['tokens']]
y_test = news_test['label']

In [22]:
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)

In [29]:
X_train_tfidf.shape

(15000, 89169)

In [ ]:
logreg_cv = GridSearchCV(LogisticRegression(),
                        param_grid={'C': [0.001, 0.01, 0.1, 1, 10]})

logreg_cv.fit(X_train_tfidf, y_train)

In [31]:
pd.DataFrame(logreg_cv.cv_results_)[["rank_test_score", "param_C", "mean_test_score", "std_test_score"]]

,rank_test_score,param_C,mean_test_score,std_test_score
0,5,0.001,0.246867,0.002696
1,4,0.01,0.725733,0.003929
2,3,0.1,0.800600,0.004683
3,2,1,0.860067,0.001511
4,1,10,0.879400,0.002489


In [33]:
svm_cv = GridSearchCV(SVC(),
                      param_grid={'C': [0.01, 0.1, 1]},
                      n_jobs=-1, cv=3)

svm_cv.fit(X_train_tfidf, y_train)

GridSearchCV(cv=3, estimator=SVC(), n_jobs=-1, param_grid={'C': [0.01, 0.1, 1]})

In [35]:
pd.DataFrame(svm_cv.cv_results_)[['rank_test_score', 'param_C', 'mean_test_score', 'std_test_score']]

,rank_test_score,param_C,mean_test_score,std_test_score
0,3,0.01,0.147667,0.000094
1,2,0.1,0.653067,0.007028
2,1,1,0.855533,0.001893


In [36]:
X_test_tfidf = tfidf.transform(X_test)

/home/vladimir/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:1208: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [41]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [46]:
for model in [logreg_cv.best_estimator_, svm_cv.best_estimator_]:
    
    y_pred = model.predict(X_test_tfidf)
    
    print(model.__class__.__name__)
    print(f'Accuracy: {accuracy_score(y_pred, y_test):.4f}')
    print(f'Precision: {precision_score(y_pred, y_test, average="weighted"):.4f}')
    print(f'Recall: {recall_score(y_pred, y_test, average="weighted"):.4f}')
    print()

LogisticRegression
Accuracy: 0.8903
Precision: 0.8940
Recall: 0.8903

SVC
Accuracy: 0.8797
Precision: 0.8933
Recall: 0.8797

